In [6]:
import pandas as pd

train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('x_test.csv')
sample_submission = pd.read_csv('sample_submission.csv')

# Display the first few rows
print(train_data.head())

# Get summary statistics
print(train_data.describe())

   id     Col 1     Col 2     Col 3     Col 4     Col 5     Col 6     Col 7  \
0   0  0.038076  0.050680  0.061696  0.021872 -0.044223 -0.034821 -0.043401   
1   1 -0.001882 -0.044642 -0.051474 -0.026328 -0.008449 -0.019163  0.074412   
2   2  0.085299  0.050680  0.044451 -0.005671 -0.045599 -0.034194 -0.032356   
3   3 -0.089063 -0.044642 -0.011595 -0.036656  0.012191  0.024991 -0.036038   
4   4  0.005383 -0.044642 -0.036385  0.021872  0.003935  0.015596  0.008142   

      Col 8     Col 9  ...    Col 56    Col 57    Col 58    Col 59    Col 60  \
0 -0.002592  0.019908  ... -0.022038 -0.031125 -0.000922  0.033494  0.000852   
1 -0.039493 -0.068330  ... -0.011564  0.012973  0.023783 -0.023815 -0.094506   
2 -0.002592  0.002864  ... -0.022063 -0.018016  0.004913  0.032956  0.018281   
3  0.034309  0.022692  ... -0.009984 -0.003373 -0.019109  0.008159  0.001898   
4 -0.002592 -0.031991  ... -0.024091 -0.026846 -0.029687  0.030984  0.014489   

     Col 61    Col 62    Col 63    Col 64   

In [7]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error
import numpy as np

# Separate features and target from training data
X = train_data.drop(columns=['id', 'y'])
y = train_data['y']

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the models
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(),
    'Lasso Regression': Lasso()
}

# Train and evaluate each model using cross-validation
scores = {}
for model_name, model in models.items():
    # Train the model
    model.fit(X_train, y_train)
    
    # Perform cross-validation
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    
    # Store the mean and standard deviation of the cross-validation scores
    scores[model_name] = (np.mean(cv_scores), np.std(cv_scores))

scores


{'Linear Regression': (-27939.328233417724, 30613.24627123543),
 'Ridge Regression': (-4288.743660336313, 1232.1729475981901),
 'Lasso Regression': (-3981.660586736045, 1285.5423163672062)}

In [8]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid for Lasso
param_grid = {
    'alpha': [0.001, 0.01, 0.1, 1, 10, 100]
}

# Initialize GridSearchCV with Lasso Regression
grid_search = GridSearchCV(Lasso(), param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)

# Fit the model to the data
grid_search.fit(X_train, y_train)

# Get the results
best_params = grid_search.best_params_
best_score = -grid_search.best_score_

best_params, best_score


/Users/mihir/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.266e+04, tolerance: 5.689e+01
  model = cd_fast.enet_coordinate_descent(
/Users/mihir/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.333e+04, tolerance: 5.719e+01
  model = cd_fast.enet_coordinate_descent(
/Users/mihir/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duali

({'alpha': 0.1}, 3079.8140170680526)

In [11]:
# Use the best model from grid search to make predictions on the test set
best_lasso_model = grid_search.best_estimator_
predictions = best_lasso_model.predict(test_data.drop(columns=['id']))

# Create the submission DataFrame
submission = test_data[['id']].copy()
submission['y'] = predictions

submission.head()

submission.to_csv("submission.csv")